In [8]:
from pathlib import Path
import os
from dotenv import load_dotenv

# LangChain Core
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# LangChain Integrations
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings

In [3]:
# Load environment
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

# Embedding function (must match what was used before)
embedding_model = OpenAIEmbeddings(openai_api_key=openai_key)

# Load persisted vector store
persist_dir = "../data/vectorstore/chroma"
vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embedding_model
)

retriever = vectorstore.as_retriever()

/var/folders/68/70gzhfw57xb4cvbq9z31v2r00000gn/T/ipykernel_32299/2993780015.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=openai_key)
/var/folders/68/70gzhfw57xb4cvbq9z31v2r00000gn/T/ipykernel_32299/2993780015.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [41]:
#Our custom prompt to shape answers.
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["question", "context"],  # <-- REQUIRED
    template="""
You are a cycle-aware nutrition assistant based on holistic and scientific insights.

Always answer user questions helpfully and always provide answers in a warm, empowering tone.

If the source materials don’t mention something, don't mention that explicitly in your answers. Never say “The text does not provide…” — instead say things like:
“There are no specific recommendations for your question in science, but what science does advice...”

Be concise, clear, and nurturing in your responses.

Answer based on the context below:

Context:
{context}

Question:
{question}
""")

In [42]:
#Set up chat memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"  # ✅ explicitly set the output key to store
)

In [43]:
#Create Conversational QA Chain (with memory)
llm = ChatOpenAI(model="gpt-4", temperature=0)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    return_source_documents=True,
    output_key="answer"  #tell memory to store just the answer (not the resource docs used): clear output key.
)

In [44]:
#tryout: ask assistant a question in a multi turn conversation
# Start with a first question
query_1 = "What nutrients help with PMS symptoms in the luteal phase?"
response_1 = qa_chain({"question": query_1})
print("💬 Q1:", query_1)
print("🧠 A1:", response_1["answer"])

# Follow up naturally (memory will preserve phase context)
query_2 = "And what should I avoid eating during that time?"
response_2 = qa_chain({"question": query_2})
print("\n💬 Q2:", query_2)
print("🧠 A2:", response_2["answer"])


💬 Q1: What nutrients help with PMS symptoms in the luteal phase?
🧠 A1: During the luteal phase, it's beneficial to increase your intake of complex carbohydrates, such as millet and sweet potatoes, to stabilize serotonin and dopamine levels and help prevent mood swings. Nutrient-dense foods that provide more calories are also recommended. Foods like cauliflower, collard greens, daikon, onion, parsnip, radish, squash, and sweet potato can be particularly beneficial. Additionally, a diet rich in calcium and vitamin D has been associated with a reduced risk for PMS. Remember, every body is unique, so it's important to listen to your own body's needs and responses to different foods.

💬 Q2: And what should I avoid eating during that time?
🧠 A2: During the luteal phase of your menstrual cycle, it's recommended to focus on consuming foods like cauliflower, collard greens, daikon, onion, parsnip, radish, squash, and sweet potato. However, to prevent bloating and other PMS symptoms, it would be

In [45]:
query = "What should I eat during ovulation?"
response = qa_chain({"question": query})
print("💡 Answer:\n", response["answer"])


💡 Answer:
 During ovulation, it's beneficial to enjoy fresh, raw foods such as vegetables, fruits, and salads. Juices and smoothies are also excellent choices. When it comes to cooking, lighter preparations like steaming or poaching are recommended. Foods that promote antioxidative well-being and provide vascular support for your ovaries, such as quinoa and corn, are also beneficial. These foods can help your body create the healthiest egg possible and ward off estrogen-driven symptoms like acne and bloating. Additionally, consuming a lot of fiber-rich foods during this phase supports your large intestine to increase transit time and help flush the estrogen that the liver is working hard to metabolize.
